In [1]:
import numpy as np
import scipy.special
import scipy.ndimage
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # structure of the NN
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # learning rate
        self.lr = learningrate
        
        # weight matrices
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # activation function: sigmoid function
        self.activation_function = lambda x:scipy.special.expit(x)
        self.inverse_activation_function = lambda x: scipy.special.logit(x)
        
    def train(self, inputs_list, targets_list):
        # convert input to 2d array
        inputs = np.array(inputs_list, ndmin = 2).T
        targets = np.array(targets_list, ndmin = 2).T
        
        # calculate the input for the hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the output of the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate the input for the output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the output of the output layer
        final_outputs = self.activation_function(final_inputs)
    
        # calculate the output layer errors
        output_errors = targets - final_outputs
        # calculate the hidden layer errors
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weights between hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1 - final_outputs)), np.transpose(hidden_outputs))

        # update the weights between input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), np.transpose(inputs))

        
    def query(self, input_list):
        # convert input to 2d array
        inputs = np.array(input_list, ndmin = 2).T
        
        # calculate the input for the hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the output of the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate the input for the output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the output of the output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    def backquery(self, targets_list):
        final_outputs = np.array(targets_list, ndmin=2).T
        
        final_inputs = self.inverse_activation_function(final_outputs)

        # calculate the signal out of the hidden layer
        hidden_outputs = np.dot(self.who.T, final_inputs)
        # scale them back to 0.01 to .99
        hidden_outputs -= np.min(hidden_outputs)
        hidden_outputs /= np.max(hidden_outputs)
        hidden_outputs *= 0.98
        hidden_outputs += 0.01
        
        # calculate the signal into the hidden layer
        hidden_inputs = self.inverse_activation_function(hidden_outputs)
        
        # calculate the signal out of the input layer
        inputs = np.dot(self.wih.T, hidden_inputs)
        # scale them back to 0.01 to .99
        inputs -= np.min(inputs)
        inputs /= np.max(inputs)
        inputs *= 0.98
        inputs += 0.01
        
        return inputs


In [3]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10


learning_rate = 0.01

n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)


In [4]:
training_data_file = open("mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()



In [5]:
epochs = 10
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        inputs_plus10 = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, reshape=False).reshape(1, 784)
        inputs_minus10 = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, reshape=False).reshape(1, 784)
        n.train(inputs_plus10, targets)
        n.train(inputs_minus10, targets)
        
        
        

In [6]:
test_data_file = open("mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

In [7]:
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (np.asfarray(all_values[1:])/ 255.0 * 0.99)
    outputs = n.query(inputs)
    label = np.argmax(outputs)
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9756
